In [1]:
from utils import create_data_loader,build_vocab
from torchtext.data.utils import get_tokenizer
from baseline import init_model

import os
from os.path import join, exists
import torch
import torch.nn as nn

import math
import time


## PARSER

In [2]:
model_choice = 'baseline'

In [3]:
if model_choice == 'baseline':
    from baseline import init_model, train, evaluate

In [4]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [5]:
def model_execute(model,train_data,val_data,test_data,criterion,optimizer,clip_value, n_epochs,models_path):
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):

        start_time = time.time()

        train_loss = train(model, train_data, optimizer, criterion, clip_value)
        valid_loss = evaluate(model, val_data, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    test_loss = evaluate(model, test_data, criterion)

    print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')
    
    torch.save(model.state_dict(),join(models_path,"model" + time.strftime("_%d-%m_%H-%M")))
    
    return model

In [6]:
NL_TOKENIZER = get_tokenizer('spacy', language='en_core_web_sm')
QL_TOKENIZER = get_tokenizer('spacy', language='en_core_web_sm')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
data_path = join("..","..","data","eiopa","3_processed")
train_file_nl = join(data_path,"data_baseline.nl")
train_file_ql = join(data_path,"data_baseline.ql")

models_path = join("..","..","models")

if exists(train_file_nl) and exists(train_file_ql):
    print("Required files exist!")
    
if exists(models_path):
    print("Model path exists!")
    
vocab_nl = build_vocab(train_file_nl,NL_TOKENIZER)

vocab_ql = build_vocab(train_file_ql,QL_TOKENIZER)

Required files exist!
Model path exists!


In [8]:
data = create_data_loader(train_file_nl,train_file_ql,vocab_nl,vocab_ql,shuffle = True)

In [9]:
model,optimizer = init_model(DEVICE,len(vocab_nl),len(vocab_ql))

Model initialized! 
 The model has     205,203 trainable parameters


In [10]:
PAD_IDX = vocab_ql.stoi['<pad>'] # for ouput language
CRITERION = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [11]:
model_trained = model_execute(model, data, data, data,CRITERION,optimizer,clip_value = 1, n_epochs = 20,models_path = models_path)

Epoch: 01 | Time: 0m 30s
	Train Loss: 4.825 | Train PPL: 124.544
	 Val. Loss: 3.577 |  Val. PPL:  35.754
Epoch: 02 | Time: 0m 29s
	Train Loss: 3.522 | Train PPL:  33.837
	 Val. Loss: 3.480 |  Val. PPL:  32.454
Epoch: 03 | Time: 0m 30s
	Train Loss: 3.303 | Train PPL:  27.195
	 Val. Loss: 3.255 |  Val. PPL:  25.917
Epoch: 04 | Time: 0m 32s
	Train Loss: 2.892 | Train PPL:  18.024
	 Val. Loss: 2.900 |  Val. PPL:  18.169
Epoch: 05 | Time: 0m 30s
	Train Loss: 2.380 | Train PPL:  10.809
	 Val. Loss: 2.658 |  Val. PPL:  14.271
Epoch: 06 | Time: 0m 30s
	Train Loss: 1.893 | Train PPL:   6.640
	 Val. Loss: 2.469 |  Val. PPL:  11.807
Epoch: 07 | Time: 0m 33s
	Train Loss: 1.577 | Train PPL:   4.839
	 Val. Loss: 2.546 |  Val. PPL:  12.757
Epoch: 08 | Time: 0m 30s
	Train Loss: 1.361 | Train PPL:   3.899
	 Val. Loss: 2.616 |  Val. PPL:  13.685
Epoch: 09 | Time: 0m 29s
	Train Loss: 1.251 | Train PPL:   3.492
	 Val. Loss: 2.672 |  Val. PPL:  14.463
Epoch: 10 | Time: 0m 30s
	Train Loss: 1.109 | Train PPL

## ---------------------------------------------------